# Get

# Maya external socket command

## 1. Maya server: run the follow command in the script editor(Python) in Maya

### open server
import maya.cmds as cmds

cmds.commandPort(n="localhost:12345")

### close server

cmds.commandPort(cl=True, n="localhost:12345")


In [1]:
import socket
import numpy as np

In [2]:
# connect to Maya server
HOST = '127.0.0.1'                 # Symbolic name meaning the local host
PORT = 12345              # Arbitrary non-privileged port

ADDR=(HOST,PORT)

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(ADDR)

In [3]:
def SendCommand(command:str):
    #client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    #client.connect(ADDR)
    command = command.encode() # the command from external editor to maya

    MyMessage = command
    client.send(MyMessage)
    data = client.recv(1024)#receive the result info
    #client.close()
    #print(data)
    #ret = str(data.decode(encoding="ASCII"))
    ret = data.decode("utf-8")
    return ret

In [4]:
#Test one: send command 
command = "polyCube()"
SendCommand(command)

'pCube1\tpolyCube1\n\x00'

In [5]:
def GetObjectWorldTransform(object_name:str):
    '''
    Get Object world location
    return: cordindate [x,y,z]
    '''
    send_message = "xform -q -t -ws " + object_name + ";"
    recv_message = SendCommand(send_message)
    recv_message = recv_message.rstrip('\x00').rstrip('\n').split('\t')
    return [np.around(float(_), decimals = 2) for _ in recv_message]

In [7]:
#Test two: get object world location
GetObjectWorldTransform("Main")
#GetObjectWorldTransform("Root_M")


[0.0, 0.0, -4.94]

In [ ]:
#['blink','translateX']

In [9]:
def GetObjectAttribute(object_name:str, attr_name:str):
    '''
    Get attribute value
    return [float]
    '''
    send_message = "getAttr " + object_name + "." + attr_name + ";"
    recv_message = SendCommand(send_message)
    recv_message = recv_message.rstrip('\x00').rstrip('\n').split('\t')
    return [np.around(float(_), decimals = 2) for _ in recv_message]

In [10]:
#Test Three: get object attribute
GetObjectAttribute("ctrlEye_L","blink")

[0.0]

In [11]:
def GetBodyInformation(joint_list : list):
    BODY_INFO_DIC = {}
    for joint_name in joint_list:
        
        info_dict = {}
        world_transform = GetObjectWorldTransform(joint_name)
        info_dict["world_transform"] = world_transform
        
        local_transform = GetObjectAttribute(joint_name, "translate")
        info_dict["local_transform"] = local_transform
        
        local_rotation = GetObjectAttribute(joint_name, "rotate")
        info_dict["local_rotation"] = local_rotation
        
        BODY_INFO_DIC[joint_name] = info_dict
    
    return BODY_INFO_DIC


In [12]:
BODY_JOINT_LIST = ["Main","FKRoot_M", "FKSpine1_M", "FKChest_M", "Neck_M",
                   "FKShoulder_R", "FKShoulder_L", "FKElbow_R", "FKElbow_L", "FKWrist_R", "FKWrist_L", "IKLeg_R", "IKLeg_L"]

In [13]:
# Test three: get body joint
body_joint_info_dict = GetBodyInformation(BODY_JOINT_LIST)

In [14]:
body_joint_info_dict

{'Main': {'world_transform': [0.0, 0.0, 0.0],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKRoot_M': {'world_transform': [0.0, 9.83, -0.62],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKSpine1_M': {'world_transform': [-0.0, 11.19, -0.42],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKChest_M': {'world_transform': [-0.0, 12.61, -0.55],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'Neck_M': {'world_transform': [-0.0, 14.12, -0.26],
  'local_transform': [1.54, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKShoulder_R': {'world_transform': [-1.32, 13.33, -0.43],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKShoulder_L': {'world_transform': [1.32, 13.33, -0.43],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKElbow_R': {'world_transform': [-3.87, 13.28, -0.56],
  'local_transform': [0.0,

In [15]:
def GetFingerCurlInformation():    
    info_dict = {"right_fingers":{}, "left_fingers":{}}
    
    thumb_curl = GetObjectAttribute("Fingers_R", "thumbCurl")
    info_dict["right_fingers"]["thumb_curl"] = thumb_curl
    
    index_curl = GetObjectAttribute("Fingers_R", "indexCurl")
    info_dict["right_fingers"]["index_curl"] = index_curl
    
    middle_curl = GetObjectAttribute("Fingers_R", "middleCurl")
    info_dict["right_fingers"]["middle_curl"] = middle_curl
    
    ring_curl = GetObjectAttribute("Fingers_R", "ringCurl")
    info_dict["right_fingers"]["ring_curl"] = ring_curl
    
    pinky_curl = GetObjectAttribute("Fingers_R", "pinkyCurl")
    info_dict["right_fingers"]["pinky_curl"] = pinky_curl
    
    thumb_curl = GetObjectAttribute("Fingers_L", "thumbCurl")
    info_dict["left_fingers"]["thumb_curl"] = thumb_curl

    index_curl = GetObjectAttribute("Fingers_L", "indexCurl")
    info_dict["left_fingers"]["index_curl"] = index_curl

    middle_curl = GetObjectAttribute("Fingers_L", "middleCurl")
    info_dict["left_fingers"]["middle_curl"] = middle_curl

    ring_curl = GetObjectAttribute("Fingers_L", "ringCurl")
    info_dict["left_fingers"]["ring_curl"] = ring_curl

    pinky_curl = GetObjectAttribute("Fingers_L", "pinkyCurl")
    info_dict["left_fingers"]["pinky_curl"] = pinky_curl
    
    return info_dict  

In [16]:
# Test four: get body joint
finger_curl_info_dict = GetFingerCurlInformation()

In [17]:
finger_curl_info_dict

{'right_fingers': {'thumb_curl': [0.0],
  'index_curl': [0.0],
  'middle_curl': [0.0],
  'ring_curl': [0.0],
  'pinky_curl': [0.0]},
 'left_fingers': {'thumb_curl': [0.0],
  'index_curl': [0.0],
  'middle_curl': [0.0],
  'ring_curl': [0.0],
  'pinky_curl': [0.0]}}

In [18]:
def GetFaceInformation():
    info_dict = {}
    eyebrow_r_x = GetObjectAttribute("ctrlBrow_R", "translateX")
    eyebrow_r_y = GetObjectAttribute("ctrlBrow_R", "translateY")
    eyebrow_l_x = GetObjectAttribute("ctrlBrow_L", "translateX")
    eyebrow_l_y = GetObjectAttribute("ctrlBrow_L", "translateY")
    
    info_dict["eye_brow"] = {"eyebrow_r_x":eyebrow_r_x, "eyebrow_r_y":eyebrow_r_y, 
                            "eyebrow_l_x":eyebrow_l_x, "eyebrow_l_y":eyebrow_l_y}
    
    eye_r_x = GetObjectAttribute("ctrlEye_R", "translateX")
    eye_r_y = GetObjectAttribute("ctrlEye_R", "translateY")
    eye_r_blink = GetObjectAttribute("ctrlEye_R", "blink")
    
    
    eye_l_x = GetObjectAttribute("ctrlEye_L", "translateX")
    eye_l_y = GetObjectAttribute("ctrlEye_L", "translateY")
    eye_l_blink = GetObjectAttribute("ctrlEye_L", "blink")
    
    info_dict["eye"] = {"eye_r_x":eye_r_x, "eye_r_y":eye_r_y, "eye_r_blink":eye_r_blink, 
                        "eye_l_x":eye_l_x, "eye_l_y":eye_l_y, "eye_l_blink":eye_l_blink}
    
    mouth_x = GetObjectAttribute("ctrlMouth_M", "translateX")
    mouth_y = GetObjectAttribute("ctrlMouth_M", "translateY")
    
    info_dict["mouth"] = {"mouth_x":mouth_x, "mouth_y":mouth_y}
    
    mouth_corner_L = GetObjectAttribute("ctrlMouthCorner_L", "translateY")
    mouth_corner_R = GetObjectAttribute("ctrlMouthCorner_R", "translateY")
    
    info_dict["mouth_corner"] = {"mouth_corner_L":mouth_corner_L, "mouth_corner_R":mouth_corner_R}
    
    return info_dict

In [19]:
# Test four: get face emotions
face_info_dict = GetFaceInformation()

In [20]:
face_info_dict

{'eye_brow': {'eyebrow_r_x': [0.0],
  'eyebrow_r_y': [0.0],
  'eyebrow_l_x': [0.0],
  'eyebrow_l_y': [0.0]},
 'eye': {'eye_r_x': [0.0],
  'eye_r_y': [0.0],
  'eye_r_blink': [0.0],
  'eye_l_x': [0.0],
  'eye_l_y': [0.0],
  'eye_l_blink': [0.0]},
 'mouth': {'mouth_x': [0.0], 'mouth_y': [0.0]},
 'mouth_corner': {'mouth_corner_L': [0.0], 'mouth_corner_R': [0.0]}}

In [ ]:
client.close()